In [170]:
import pandas as pd
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# df = pd.read_csv('../custom_data/metric_data/final_job_list.csv')


In [3]:
# uj_map = pd.read_csv('../custom_data/metric_data/user_job_mapping.csv')

In [36]:
df =pd.read_csv('../custom_data/metric_data/analysis_set.csv')

In [28]:
# jd_list = list(df.job_id)

In [29]:
# uj_map.job_id.nunique()

In [30]:
# uj_map.user_id.nunique()

In [31]:
# main_df = uj_map.loc[uj_map['job_id'].isin(jd_list)]

In [32]:
# main_df.job_id.nunique()

In [33]:
# main_df.user_id.nunique()

In [34]:
df

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [13]:
df.job_id.nunique()

2949

In [15]:
df.user_id.nunique()

30363

In [89]:
import requests

def fetch_jobs(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    

    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    # print(response.text)
    

    # url = url.replace("<DAYS>",str(last_k)).replace("<SIZE>",str(size))

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()
    # print(job_data)
    job_df = pd.DataFrame(job_data)

    return job_df[['id','introText']]

In [90]:

jd = fetch_jobs('1373869')

In [91]:
jd

,id,introText
0,1373869,<p>Sr. Brand Marketing Manager </p><p><br/></p...


In [92]:
# df2 = df.head(100)

In [93]:
# df2

In [95]:
df.job_id.nunique()

2949

In [96]:
df.user_id.nunique()

30363

In [98]:
from tqdm import tqdm
df_list = []
for i in tqdm(df['job_id'].unique()):
    try:
        j = fetch_jobs(str(i))
    
        df_list.append(j)
    except:
        continue
jd = pd.concat(df_list)    

    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2949/2949 [00:40<00:00, 72.66it/s]


In [99]:
jd

,id,introText
0,1373681,<p>About Company - Global recognized leader in...
0,1373682,<p>About Company - Global recognized leader in...
0,1373683,<p>A prominent global investment and technolog...
0,1373684,<p>Limese is looking for a detail-oriented and...
0,1373685,<p>The RM shall be responsible for originating...
...,...,...
0,1376446,<p>Digital Analytics - Domain Specialist <br/>...
0,1376758,<p>Urban Company is a tech-enabled home servic...
0,1375146,<p>Reinsurance Manager - General Insurance </p...
0,1376759,<p>Experience in managing Architects Developer...


In [101]:
# 2949-2824

In [102]:
# d.columns = ['job_id','introText']

In [106]:
jid_list = list(jd.id)

In [110]:
# df

In [111]:
df1 = df.loc[df['job_id'].isin(jid_list)]

In [113]:
jd.columns = ['job_id','introText']

In [114]:
jd

,job_id,introText
0,1373681,<p>About Company - Global recognized leader in...
0,1373682,<p>About Company - Global recognized leader in...
0,1373683,<p>A prominent global investment and technolog...
0,1373684,<p>Limese is looking for a detail-oriented and...
0,1373685,<p>The RM shall be responsible for originating...
...,...,...
0,1376446,<p>Digital Analytics - Domain Specialist <br/>...
0,1376758,<p>Urban Company is a tech-enabled home servic...
0,1375146,<p>Reinsurance Manager - General Insurance </p...
0,1376759,<p>Experience in managing Architects Developer...


In [115]:
merged_df = pd.merge(jd, df1, on='job_id', how='left')


In [120]:
merged_df.sample(5)

,job_id,introText,application_id,user_id
81566,1375214,<p>AVP: Enterprise Business Development <br/><...,144282676,2788606
9610,1373836,<p>Senior Strategic Account Manager </p><p><br...,144307953,1287798
61741,1374791,<p>Leading e commerce market place requires Sr...,144314628,255906
25365,1374032,<p>The position is required for one of our cli...,144191649,1721447
95013,1375550,<p>Experience: At least min 4+ years post qual...,144320075,1286631


In [126]:
uid_list = list(merged_df.user_id.unique())

In [132]:
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string
     

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)

    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()
    
            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'resume' : []
    }

    for s in tqdm(user_json):

        try:
            source = s[0]

            user_dict['id'].append(source['id'])
            user_dict['resume'].append(clean_space(source['resumeText']))

        except Exception as e:
            continue

    return user_dict

In [133]:

def get_user_data_in_batches(uid_list, batch_size=1000):
    user_data = pd.DataFrame()

    for i in range(0, len(uid_list), batch_size):
        batch_uids = uid_list[i:i + batch_size]

        batch_user_data = get_user_data_search_embed(batch_uids)

        user_data = pd.concat([user_data, batch_user_data], ignore_index=True)

    return user_data

In [134]:
batched_user_data = get_user_data_in_batches(uid_list, batch_size=1000)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 399/399 [00:00<00:00, 4480.42it/s]


In [136]:
len(uid_list)

29399

In [137]:
batched_user_data.columns = ['user_id','resume']

In [145]:
batched_user_data.dtypes

user_id    object
resume     object
dtype: object

In [146]:
batched_user_data['user_id']=batched_user_data['user_id'].astype(int)

In [147]:
main_df=pd.merge(batched_user_data, merged_df, on='user_id', how='left')

In [151]:
main_df.sample(5)

,user_id,resume,job_id,introText,application_id
86372,1313207,"Anita Singh Gurgaon, anitasinghmail@gmail.com ...",1375937,<p>Key Responsibilities :<br/><br/>- Strategic...,144363980
71025,1363054,"PUSHPENDRA MEENA pushpendra.m19@fms.edu Male, ...",1373849,<p>Urban Company is a technology platform offe...,144210156
41729,177956,"VISHAL BHATIA, vishal.d.bhatia@gmail.com, +91-...",1374875,<p>Product Head </p><p><br/></p><p>Job Title: ...,144425989
65244,1084020,ASHISH PANT Summary Service-driven professiona...,1373833,<p>Head EBO<br/><br/>Domnic Lewis has been man...,144184841
67664,385404,"CA Saumitra Bhat Chartered Accountant, India, ...",1376206,<p>Alert from 4S Advisory (www.4sadvisory.com)...,144399298


In [154]:
main_df.isnull().sum()

user_id           0
resume            0
job_id            0
introText         0
application_id    0
dtype: int64

In [171]:
def clean_text(input_text):
    
    '''Make text lowercase, remove text in square brackets,remove links,
    remove punctuation and remove words containing numbers.'''

    # This code block was added to remove the company names from Job embeddings
    
    # t = input_text.split('-')
    # if len(t)>3:
    #     t1 = t[1:]
    #     text = ' '.join(t1)
    # else:
    #     text = input_text
    #     pass

    text = input_text
        
    text = str(text).lower()
    text = re.sub('\[.*?\]',' ', text)
    text = re.sub('https?://\S+|www\.\S+',' ', text)
    text = re.sub('<.*?>+',' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation),' ', text)
    text = re.sub('\n',' ', text)
    text = re.sub('\w*\d\w*',' ', text)
    text = re.sub('yrs',' ',text)
    text = re.sub('\r',' ',text)
    text = ' '.join([word for word in text.split() if word not in (stop)])
    text = re.sub(' +', ' ', text)
    
    return text

In [172]:
li = []
for d in tqdm(main_df['introText']):
    des = clean_text(d)
    li.append(des)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 126362/126362 [01:37<00:00, 1294.10it/s]


In [173]:
main_df['description']=li

In [174]:
main_df

,user_id,resume,job_id,introText,application_id,description
0,439042,Vibha Issar Email: vibhaissar@rediffmail.com M...,1373681,<p>About Company - Global recognized leader in...,144274567,company global recognized leader financial tec...
1,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...,1373681,<p>About Company - Global recognized leader in...,144261479,company global recognized leader financial tec...
2,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...,1373729,<p>- MBA in Corporate Finance or related field...,144261360,mba corporate finance related field years expe...
3,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...,1373732,<p>A Mumbai-based family office is looking to ...,144261352,mumbai based family office looking hire equity...
4,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...,1373753,<p>The Business Finance Partner is a full-time...,144261337,business finance partner full time role respon...
...,...,...,...,...,...,...
126357,1984167,Contact Details: Email: bvkarthik88@gmail.com ...,1376745,<p><b>Role : Director - Compliance Investigato...,144441737,role director compliance investigator financia...
126358,770660,Profile Dynamic leader and a Certified Risk Ma...,1376742,<p><b>Role : AVP/Senior Manager - Ethics and C...,144438628,role avp senior manager ethics compliance mana...
126359,2747092,ANGAD KUMAR MISHRA Contact: 9990897791 Email: ...,1376747,<p><b>Role : Senior Manager - HR Compliance & ...,144440377,role senior manager hr compliance labour relat...
126360,138055,SIDDHARTH ARORA Post Graduate Diploma in Manag...,1376729,<p>Executive Hiring is looking for a Head - Su...,144439177,executive hiring looking head supply chain pla...


In [165]:
# main_df[['resume']].values

In [176]:
# main_df.to_csv('../custom_data/metric_data/main_data.csv', index=False)